In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 123.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.6 MB/s eta 0:00:00


In [ ]:
import gradio as gr

In [ ]:
# connecting google drive to colab
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from peft import PeftModel, PeftConfig
import torch

# Step 1: Paths
adapter_path = "/content/drive/My Drive/Deakin_units/T1-2025/SIT764/sentiment-analysis/flan-t5-large/sentiment_model/session_20250509_021444/"
base_model_name = "google/flan-t5-large"  # same as used during training

# Step 2: Load base model and tokenizer
tokenizer = T5Tokenizer.from_pretrained(base_model_name)
base_model = T5ForConditionalGeneration.from_pretrained(base_model_name)

# Step 3: Load the adapter
model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()

# Step 4: Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.encoder.block.0.layer.0.SelfAttention.q.lora_A.default.weight', 'base_model.model.encoder.block.0.layer.0.SelfAttention.q.lora_B.default.weight', 'base_model.model.encoder.block.0.layer.0.SelfAttention.v.lora_A.default.weight', 'base_model.model.encoder.block.0.layer.0.SelfAttention.v.lora_B.default.weight', 'base_model.model.encoder.block.1.layer.0.SelfAttention.q.lora_A.default.weight', 'base_model.model.encoder.block.1.layer.0.SelfAttention.q.lora_B.default.weight', 'base_model.model.encoder.block.1.layer.0.SelfAttention.v.lora_A.default.weight', 'base_model.model.encoder.block.1.layer.0.SelfAttention.v.lora_B.default.weight', 'base_model.model.encoder.block.2.layer.0.SelfAttention.q.lora_A.default.weight', 'base_model.model.encoder.block.2.layer.0.SelfAttention.q.lora_B.default.weight', 'base_model.model.encoder.block.2.layer.0.Sel

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=32, out_features=384, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()


In [ ]:
# Step 5: Label mapping (assuming textual generation output, not class logits)
valid_labels = ["negative", "neutral", "positive"]

def predict_sentiment(text):
    prompt = f"Classify the sentiment of the following text as positive, negative, or neutral: {text}. Output only one of these labels."
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)

    with torch.no_grad():
        output_ids = model.generate(**inputs, max_new_tokens=5)
        output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip().lower()

        # Normalize to valid label
        for label in valid_labels:
            if label in output_text:
                return label.capitalize()

        return "Unknown"

In [ ]:
print(predict_sentiment("I did not like this drug at all. I hated it. It had too many side effects and I got a headache also."))

Negative


In [ ]:
print(predict_sentiment("I love this drug"))

Positive


In [ ]:
iface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(label="Enter Review"),
    outputs=gr.Textbox(label="Predicted Sentiment"),
    title="Sentiment Classifier (flan-t5-large)",
    description="This UI classifies sentiment using a fine-tuned FLAN-T5-Large model."
)

In [ ]:
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://35ad2a05a2c809aba7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
